In [2]:
import cv2
import numpy as np
import time

def initialize_webcam():
    cap = cv2.VideoCapture(0)  # Use 0 for default webcam
    if not cap.isOpened():
        print("Error: Could not open webcam.")
    return cap

def count_fingers(frame, threshold=50):
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Apply Gaussian blur
    blur = cv2.GaussianBlur(gray, (35, 35), 0)
    # Apply threshold
    _, thresh = cv2.threshold(blur, threshold, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    # Find contours
    contours, _ = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) == 0:
        return frame, 0
    
    # Find the contour with the maximum area
    max_contour = max(contours, key=lambda x: cv2.contourArea(x))
    
    # Create a convex hull around the hand
    hull = cv2.convexHull(max_contour)
    
    # Find convexity defects
    hull_indices = cv2.convexHull(max_contour, returnPoints=False)
    defects = cv2.convexityDefects(max_contour, hull_indices)
    
    if defects is None:
        return frame, 0
    
    # Draw the contours
    cv2.drawContours(frame, [max_contour], -1, (0, 255, 0), 2)
    cv2.drawContours(frame, [hull], -1, (0, 0, 255), 2)
    
    # Find the center of the palm
    moments = cv2.moments(max_contour)
    if moments['m00'] != 0:
        center_x = int(moments['m10'] / moments['m00'])
        center_y = int(moments['m01'] / moments['m00'])
    else:
        center_x, center_y = 0, 0
    center = (center_x, center_y)
    
    finger_count = 0
    
    for i in range(defects.shape[0]):
        start_idx, end_idx, far_idx, _ = defects[i, 0]
        start = tuple(max_contour[start_idx][0])
        end = tuple(max_contour[end_idx][0])
        far = tuple(max_contour[far_idx][0])
        
        # Calculate the angle
        a = np.linalg.norm(np.array(start) - np.array(end))
        b = np.linalg.norm(np.array(start) - np.array(far))
        c = np.linalg.norm(np.array(end) - np.array(far))
        angle = np.arccos((b**2 + c**2 - a**2) / (2 * b * c)) * (180 / np.pi)
        
        # If the angle is less than 90 degrees, consider it as a finger
        if angle <= 90:
            finger_count += 1
            cv2.line(frame, center, start, (255, 0, 0), 2)
            cv2.line(frame, center, end, (255, 0, 0), 2)
            cv2.circle(frame, start, 8, (0, 0, 255), -1)
            cv2.circle(frame, end, 8, (0, 0, 255), -1)
    
    return frame, finger_count

def detect_fingers():
    cap = initialize_webcam()
    if not cap:
        return

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("Error: Failed to capture image.")
                break

            # Detect fingers and draw lines
            frame, finger_count = count_fingers(frame)

            # Display the number of fingers
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(frame, f'Fingers: {finger_count}', (10, 30), font, 1, (255, 0, 0), 2, cv2.LINE_AA)

            # Display the frame
            cv2.imshow('Real-Time Finger Detection', frame)

            # Exit on ESC key
            if cv2.waitKey(1) & 0xFF == 27:
                break

            # Optional: Pause for a short time between frames
            time.sleep(0.1)

    finally:
        cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    detect_fingers()
